In [1]:
import pandas as pd
import numpy as np
import glob
import gc
import keras 
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, MaxPooling2D, Conv2D, Input
from keras.layers import Flatten, BatchNormalization, Activation, Reshape, concatenate
from keras.models import Model
import random
from sklearn.model_selection import KFold
import keras.backend as K
from sklearn.utils import class_weight

gc.enable()

Using TensorFlow backend.


In [2]:
spectrums_index = np.array(glob.glob('E:/notes_database/spectrograms_3d/*'))
labels_index = np.array(glob.glob('../preparations/labels/*'))
kf = KFold(n_splits=12, random_state=7)

In [3]:
num_samples = 5000
height = 60
width = 108
channels = 2
steps = 20
spec_list = spectrums_index[:num_samples]
labe_list = labels_index[:num_samples]

In [4]:
train_order = []
test_order = []
length = np.arange(num_samples)

for x, y in kf.split(range(num_samples)):
    train_order.append(length[x])
    test_order.append(length[y])
train_order = np.array(train_order)
test_order = np.array(test_order)

In [5]:
test_order[0].shape

(417,)

In [6]:
def inputs_to_model(spectrum_list, labels_list, order, num, start, ending):
    lstm_output = []
    cnn_output = []
    y_train = []
    
    import time
    start_ = time.time()

    for file in order[num][start:ending]:
        spectrum = np.load(spectrum_list[file])
        labels = np.load(labels_list[file])
        
        _lstm = np.zeros((20, 24))
        for count, i in enumerate(labels):
            _cnn = spectrum[:, 27*count:108+27*count, :]
            if _cnn.shape == (60, 108, channels):
                lstm_output.append(_lstm)
                cnn_output.append(_cnn)
                y_train.append(i)
                
            _lstm = _lstm[1:, :]
            _lstm = np.append(_lstm, i.reshape((1, 24)), axis=0)
        
    lstm_inputs = np.array(lstm_output)
    cnn_inputs = np.array(cnn_output)
    labels_inputs = np.array(y_train)
    print(time.time()-start_)
    return lstm_inputs, cnn_inputs, labels_inputs

In [7]:
lstm_test, cnn_test, labels_test = inputs_to_model(spec_list, labe_list, test_order, 0, 0, test_order[0].shape[0]+1)

31.352017164230347


In [8]:
def train_test_whole(classifier, num, name, spectrum_list, labels_list, epochs=100, batchsize=32):
    lstm_inputs, cnn_inputs, labels_inputs = inputs_to_model(spectrum_list, labels_list, train_order, 
                                                             num, 0, train_order[num].shape[0]+1)

    classifier.fit([lstm_inputs, cnn_inputs], labels_inputs, epochs=epochs, batch_size=batchsize, verbose=1,
                  validation_data = ([lstm_test, cnn_test], labels_test))
    
    classifier.save_weights(name, overwrite=True)   
    del classifier
    K.clear_session()

In [14]:
def train_test_part(classifier, num, name, spectrum_list, labels_list, epochs=100, batchsize=32, split=7):
    
    num_samples = train_order[num].shape[0]
    
    for j in range(epochs):
        for i in range(split):
            lstm_inputs, cnn_inputs, labels_inputs = inputs_to_model(spectrum_list, labels_list, train_order, 
                                                                     num, num_samples//split*i, num_samples//split*(i+1))

            classifier.fit([lstm_inputs, cnn_inputs], labels_inputs, epochs=1, batch_size=batchsize, verbose=1,
                          validation_data = ([lstm_test, cnn_test], labels_test))
            gc.collect()
    
    classifier.save_weights(name, overwrite=True)   
    del classifier
    K.clear_session()

In [10]:
def classifier_1(optimizer='adam', loss='binary_crossentropy', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, channels), name='cnn_inputs')
    layers = BatchNormalization()(cnn_inputs)
    layers = Conv2D(32, (3,3), padding="same", activation="relu")(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(16)(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([lstm_layers, layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[lstm_inputs, cnn_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [13]:
adam = keras.optimizers.Adam(lr=0.005)
classifier = classifier_1(optimizer=adam)
train_test_part(classifier, 0, 'weights/weight-cnn3d-model1.hdf5', spec_list, labe_list, epochs=15, split=10)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':


35.91341853141785
Train on 56374 samples, validate on 51317 samples
Epoch 1/1
56374/56374 [==============================] - 125s 2ms/step - loss: 0.0612 - acc: 0.9791 - val_loss: 0.0498 - val_acc: 0.9824
47.82452726364136
Train on 56397 samples, validate on 51317 samples
Epoch 1/1
56397/56397 [==============================] - 139s 2ms/step - loss: 0.0423 - acc: 0.9836 - val_loss: 0.0380 - val_acc: 0.9851
41.583988428115845
Train on 56390 samples, validate on 51317 samples
Epoch 1/1
56390/56390 [==============================] - 118s 2ms/step - loss: 0.0380 - acc: 0.9851 - val_loss: 0.0355 - val_acc: 0.9856
37.57082486152649
Train on 56402 samples, validate on 51317 samples
Epoch 1/1
56402/56402 [==============================] - 118s 2ms/step - loss: 0.0354 - acc: 0.9859 - val_loss: 0.0320 - val_acc: 0.9870
42.402684926986694
Train on 56374 samples, validate on 51317 samples
Epoch 1/1
56374/56374 [==============================] - 114s 2ms/step - loss: 0.0342 - acc: 0.9864 - val_loss

56352/56352 [==============================] - 114s 2ms/step - loss: 0.0243 - acc: 0.9900 - val_loss: 0.0223 - val_acc: 0.9906
48.9572594165802
Train on 56374 samples, validate on 51317 samples
Epoch 1/1
56374/56374 [==============================] - 117s 2ms/step - loss: 0.0242 - acc: 0.9899 - val_loss: 0.0241 - val_acc: 0.9899
37.16312575340271
Train on 56397 samples, validate on 51317 samples
Epoch 1/1
56397/56397 [==============================] - 115s 2ms/step - loss: 0.0241 - acc: 0.9899 - val_loss: 0.0225 - val_acc: 0.9906
39.53759431838989
Train on 56390 samples, validate on 51317 samples
Epoch 1/1
56390/56390 [==============================] - 116s 2ms/step - loss: 0.0243 - acc: 0.9900 - val_loss: 0.0226 - val_acc: 0.9906
36.66326880455017
Train on 56402 samples, validate on 51317 samples
Epoch 1/1
56402/56402 [==============================] - 110s 2ms/step - loss: 0.0237 - acc: 0.9902 - val_loss: 0.0221 - val_acc: 0.9908
45.2571702003479
Train on 56374 samples, validate on 5

56352/56352 [==============================] - 108s 2ms/step - loss: 0.0222 - acc: 0.9906 - val_loss: 0.0215 - val_acc: 0.9911
61.79651141166687
Train on 56374 samples, validate on 51317 samples
Epoch 1/1
56374/56374 [==============================] - 108s 2ms/step - loss: 0.0221 - acc: 0.9908 - val_loss: 0.0220 - val_acc: 0.9911
41.988269567489624
Train on 56397 samples, validate on 51317 samples
Epoch 1/1
56397/56397 [==============================] - 107s 2ms/step - loss: 0.0223 - acc: 0.9907 - val_loss: 0.0233 - val_acc: 0.9902
62.39041471481323
Train on 56390 samples, validate on 51317 samples
Epoch 1/1
56390/56390 [==============================] - 107s 2ms/step - loss: 0.0221 - acc: 0.9907 - val_loss: 0.0201 - val_acc: 0.9916
36.19110035896301
Train on 56402 samples, validate on 51317 samples
Epoch 1/1
56402/56402 [==============================] - 106s 2ms/step - loss: 0.0217 - acc: 0.9908 - val_loss: 0.0219 - val_acc: 0.9906
39.87445783615112
Train on 56374 samples, validate o

56352/56352 [==============================] - 107s 2ms/step - loss: 0.0213 - acc: 0.9912 - val_loss: 0.0197 - val_acc: 0.9917
60.267000675201416
Train on 56374 samples, validate on 51317 samples
Epoch 1/1
56374/56374 [==============================] - 107s 2ms/step - loss: 0.0212 - acc: 0.9911 - val_loss: 0.0197 - val_acc: 0.9917
38.55345845222473
Train on 56397 samples, validate on 51317 samples
Epoch 1/1
56397/56397 [==============================] - 107s 2ms/step - loss: 0.0212 - acc: 0.9911 - val_loss: 0.0201 - val_acc: 0.9917
43.0992374420166
Train on 56390 samples, validate on 51317 samples
Epoch 1/1
56390/56390 [==============================] - 107s 2ms/step - loss: 0.0213 - acc: 0.9912 - val_loss: 0.0218 - val_acc: 0.9908
38.58469223976135
Train on 56402 samples, validate on 51317 samples
Epoch 1/1
56402/56402 [==============================] - 120s 2ms/step - loss: 0.0209 - acc: 0.9911 - val_loss: 0.0225 - val_acc: 0.9903
50.61311340332031
Train on 56374 samples, validate on

31524

In [25]:
rmsprop = keras.optimizers.RMSprop()
classifier = classifier_1(optimizer=rmsprop)
train_test_whole(classifier, 0, 'weights/weight-75overlap-model1-rmsprop.hdf5', spec_list, labe_list, epochs=10)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':


82.68173003196716
Train on 81980 samples, validate on 41105 samples
Epoch 1/10
81980/81980 [==============================] - 197s 2ms/step - loss: 0.0643 - acc: 0.9780 - val_loss: 0.0666 - val_acc: 0.9804
Epoch 2/10
81980/81980 [==============================] - 172s 2ms/step - loss: 0.0410 - acc: 0.9838 - val_loss: 0.0827 - val_acc: 0.9806
Epoch 3/10
  160/81980 [..............................] - ETA: 1:18:13 - loss: 0.0332 - acc: 0.9859

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.120193). Check your callbacks.
  % delta_t_median)


81980/81980 [==============================] - 292s 4ms/step - loss: 0.0373 - acc: 0.9852 - val_loss: 0.0430 - val_acc: 0.9847
Epoch 4/10
81980/81980 [==============================] - 287s 4ms/step - loss: 0.0348 - acc: 0.9862 - val_loss: 0.0330 - val_acc: 0.9868
Epoch 5/10
81980/81980 [==============================] - 292s 4ms/step - loss: 0.0334 - acc: 0.9867 - val_loss: 0.0332 - val_acc: 0.9864
Epoch 6/10
 6368/81980 [=>............................] - ETA: 4:16 - loss: 0.0324 - acc: 0.9871

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\IPython\core\interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-25-b042a7f6a73b>", line 3, in <module>
    train_test_whole(classifier, 0, 'weights/weight-75overlap-model1-rmsprop.hdf5', spec_list, labe_list, epochs=10)
  File "<ipython-input-23-e4086b015576>", line 6, in train_test_whole
    validation_data = ([lstm_test, cnn_test], labels_test))
  File "c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\engine\training.py", line 1705, in fit
    validation_steps=validation_steps)
  File "c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\engine\training.py", line 1235, in _fit_loop
    outs = f(ins_batch)
  File "c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\backend\tensorflow_backend.py", line 2478, in __call__
    **self.session_kwargs)
  File "c:\us

KeyboardInterrupt: 

In [15]:
def classifier_3(optimizer='adam', loss='binary_crossentropy', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, channels), name='cnn_inputs')
    layers = Conv2D(64, (3,3), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(16)(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([lstm_layers, layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[lstm_inputs, cnn_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [12]:
adam = keras.optimizers.Adam(lr=0.005)
classifier = classifier_3(optimizer=adam)
train_test_whole(classifier, 0, 'weights/weight-75overlap-model3.hdf5', spec_list, labe_list, epochs=5)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  


52.61816191673279
Train on 81980 samples, validate on 41105 samples
Epoch 1/5
81980/81980 [==============================] - 192s 2ms/step - loss: 0.0537 - acc: 0.9806 - val_loss: 0.0477 - val_acc: 0.9834
Epoch 2/5
81980/81980 [==============================] - 373s 5ms/step - loss: 0.0371 - acc: 0.9854 - val_loss: 0.0383 - val_acc: 0.9844
Epoch 3/5
81980/81980 [==============================] - 257s 3ms/step - loss: 0.0331 - acc: 0.9866 - val_loss: 0.0314 - val_acc: 0.9874
Epoch 4/5
81980/81980 [==============================] - 285s 3ms/step - loss: 0.0308 - acc: 0.9874 - val_loss: 0.0348 - val_acc: 0.9860
Epoch 5/5
81980/81980 [==============================] - 242s 3ms/step - loss: 0.0288 - acc: 0.9881 - val_loss: 0.0307 - val_acc: 0.9875


15762

In [16]:
adam = keras.optimizers.Adam(lr=0.005)
classifier = classifier_3(optimizer=adam)
train_test_part(classifier, 0, 'weights/weight-cnn3d-model3.hdf5', spec_list, labe_list, epochs=15, split=10)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  


50.80060887336731
Train on 56374 samples, validate on 51317 samples
Epoch 1/1
56374/56374 [==============================] - 102s 2ms/step - loss: 0.0590 - acc: 0.9792 - val_loss: 0.0457 - val_acc: 0.9832
48.01833915710449
Train on 56397 samples, validate on 51317 samples
Epoch 1/1
56397/56397 [==============================] - 127s 2ms/step - loss: 0.0403 - acc: 0.9843 - val_loss: 0.0370 - val_acc: 0.9857
47.20769906044006
Train on 56390 samples, validate on 51317 samples
Epoch 1/1
56390/56390 [==============================] - 137s 2ms/step - loss: 0.0368 - acc: 0.9855 - val_loss: 0.0369 - val_acc: 0.9850
46.05168914794922
Train on 56402 samples, validate on 51317 samples
Epoch 1/1
56402/56402 [==============================] - 120s 2ms/step - loss: 0.0344 - acc: 0.9863 - val_loss: 0.0323 - val_acc: 0.9869
40.14682626724243
Train on 56374 samples, validate on 51317 samples
Epoch 1/1
56374/56374 [==============================] - 132s 2ms/step - loss: 0.0330 - acc: 0.9869 - val_loss: 

56352/56352 [==============================] - 118s 2ms/step - loss: 0.0238 - acc: 0.9902 - val_loss: 0.0276 - val_acc: 0.9881
57.51612591743469
Train on 56374 samples, validate on 51317 samples
Epoch 1/1
56374/56374 [==============================] - 110s 2ms/step - loss: 0.0240 - acc: 0.9899 - val_loss: 0.0216 - val_acc: 0.9907
37.600547552108765
Train on 56397 samples, validate on 51317 samples
Epoch 1/1
56397/56397 [==============================] - 119s 2ms/step - loss: 0.0240 - acc: 0.9899 - val_loss: 0.0216 - val_acc: 0.9910
41.146591901779175
Train on 56390 samples, validate on 51317 samples
Epoch 1/1
56390/56390 [==============================] - 110s 2ms/step - loss: 0.0238 - acc: 0.9901 - val_loss: 0.0226 - val_acc: 0.9905
64.15681409835815
Train on 56402 samples, validate on 51317 samples
Epoch 1/1
56402/56402 [==============================] - 113s 2ms/step - loss: 0.0236 - acc: 0.9900 - val_loss: 0.0287 - val_acc: 0.9887
42.583752393722534
Train on 56374 samples, validate

56352/56352 [==============================] - 119s 2ms/step - loss: 0.0224 - acc: 0.9907 - val_loss: 0.0218 - val_acc: 0.9911
81.10596132278442
Train on 56374 samples, validate on 51317 samples
Epoch 1/1
56374/56374 [==============================] - 126s 2ms/step - loss: 0.0221 - acc: 0.9907 - val_loss: 0.0217 - val_acc: 0.9911
40.9621684551239
Train on 56397 samples, validate on 51317 samples
Epoch 1/1
56397/56397 [==============================] - 149s 3ms/step - loss: 0.0223 - acc: 0.9906 - val_loss: 0.0218 - val_acc: 0.9906
44.28649020195007
Train on 56390 samples, validate on 51317 samples
Epoch 1/1
56390/56390 [==============================] - 138s 2ms/step - loss: 0.0223 - acc: 0.9908 - val_loss: 0.0222 - val_acc: 0.9908
40.068692207336426
Train on 56402 samples, validate on 51317 samples
Epoch 1/1
56402/56402 [==============================] - 135s 2ms/step - loss: 0.0219 - acc: 0.9908 - val_loss: 0.0222 - val_acc: 0.9908
43.70849084854126
Train on 56374 samples, validate on

KeyboardInterrupt: 

In [14]:
def classifier_4(optimizer='adam', loss='binary_crossentropy', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, channels), name='cnn_inputs')
    layers = MaxPooling2D(2,2, border_mode='same')(cnn_inputs)
    layers = Conv2D(32, (3,3), padding="same", activation="relu")(layers)
    layers = BatchNormalization()(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(16)(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([lstm_layers, layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[lstm_inputs, cnn_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [15]:
adam = keras.optimizers.Adam(lr=0.005)
classifier = classifier_4(optimizer=adam)
train_test_whole(classifier, 0, 'weights/weight-75overlap-model4.hdf5', spec_list, labe_list, epochs=5)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  after removing the cwd from sys.path.
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  


64.38756227493286
Train on 81980 samples, validate on 41105 samples
Epoch 1/5
81980/81980 [==============================] - 169s 2ms/step - loss: 0.0621 - acc: 0.9788 - val_loss: 0.0470 - val_acc: 0.9823
Epoch 2/5
81980/81980 [==============================] - 207s 3ms/step - loss: 0.0438 - acc: 0.9832 - val_loss: 0.0407 - val_acc: 0.9839
Epoch 3/5
81980/81980 [==============================] - 169s 2ms/step - loss: 0.0393 - acc: 0.9847 - val_loss: 0.0355 - val_acc: 0.9858
Epoch 4/5
18368/81980 [=====>........................] - ETA: 1:49 - loss: 0.0375 - acc: 0.9853

KeyboardInterrupt: 